## Data Generation with ChatGPT

In [21]:
import os
import base64
import numpy as np
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key="api-key")  # Replace with your key


In [23]:
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [25]:
def act_with_images(image_paths):
    images_payload = []
    for path in image_paths:
        base64_image = encode_image_to_base64(path)
        images_payload.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a participant in a visual decision-making game. In each trial, you are shown two boxes: one blue and one orange. "
                    "Your goal is to choose one of them. A happy face image means you received a coin (a reward) in that past trial, and a sad face means you received nothing. "
                    "You can see feedback images from up to 10 previous trials. Use this visual feedback to inform your current choice. "
                    "Respond with exactly one word: 'blue' or 'orange'."
                )
            },
            {
                "role": "user",
                "content": images_payload
            }
        ],
        max_tokens=1
    )
    #print(completion)
    return completion.choices[0].message.content.strip().lower()

In [27]:
action_to_index = {"blue": 0, "orange": 1}
num_runs = 200
num_trials = 100
reward_probs = 0.25

# File paths
box_image_path = "./image/input_image/box_images_2.png"
happy_image_path = "./image/input_image/happy_face.png"
sad_image_path = "./image/input_image/sad_face.png"

In [33]:
from datetime import datetime

a = datetime.now()

for run in range(174, num_runs + 1):
    print(f'Run {run}')
    previous_feedback_images = []
    data = []

    for trial in range(1, num_trials + 1):
        #print(trial)
        image_list = [box_image_path] + previous_feedback_images[-10:]
        
        
        #print(image_list)
        while True:
            action = act_with_images(image_list)
            if action in action_to_index:
                index_action = action_to_index[action]
                break
            else:
                print("Invalid action received. Retrying...")

        reward = np.random.binomial(1, reward_probs)
        feedback_image = happy_image_path if reward else sad_image_path
        previous_feedback_images.append(feedback_image)
        row = [run, trial, index_action, reward, reward_probs, reward_probs]
        data.append(row)
        
    df = pd.DataFrame(data, columns=['run', 'trial', 'action', 'reward', 'probs_blue', 'probs_orange'])
    output_dir = 'img_data/gpt/img_gpt-4-vision'
    os.makedirs(output_dir, exist_ok=True)
    df.to_csv(f'{output_dir}/experiment_{run}.csv', index=False)
    
b = datetime.now()
print('total_time')
print(b-a)

Run 174
Run 175
Run 176
Run 177
Run 178
Run 179
Run 180
Run 181
Run 182
Run 183
Run 184
Run 185
Run 186
Run 187
Run 188
Run 189
Run 190
Run 191
Run 192
Run 193
Run 194
Run 195
Run 196
Run 197
Run 198
Run 199
Run 200
total_time
1:21:18.285180
